<a href="https://colab.research.google.com/github/MichalWlodkowski/US_speeches/blob/master/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install scrapy
from scrapy import Selector
import requests
import numpy as np
import pandas as pd
import re

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize

     |████████████████████████████████| 245kB 2.8MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 235kB 9.0MB/s 
     |████████████████████████████████| 2.7MB 11.9MB/s 
     |████████████████████████████████| 3.2MB 27.4MB/s 
     |████████████████████████████████| 3.1MB 47.7MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7765 sha256=610de8b029f74eb1f674fbdeae8b39285d0e0387852ce7b8f38df7003689b988
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11515 sha256=6728e61ca17adad15890badc8f1f5510137322f6cb1a811f687cb14bf5b0ede4
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
Successfully built protego PyDispatcher


In [0]:
us_all = 'https://en.wikisource.org/wiki/Portal:Inaugural_Speeches_by_United_States_Presidents'

In [0]:
## Getting links to all presidential Inaugural Addresses in US

html = requests.get(us_all).content
sel = Selector( text = html )
xpath3 = '//@href'
d = pd.DataFrame(sel.xpath(xpath3).extract())
speech = d[d[0].str.contains('Inaugural_Address')].reset_index(drop = True)
speech.columns = ['links']

In [7]:
# Function to get president name from links + receiving full link for speeches
def frag(x):
    wh = x[6:x.find('%')].replace('_',' ')
    return wh
speech['president'] = speech['links'].apply(frag)
speech['full_links'] = speech['links'].apply(lambda x: 'https://en.wikisource.org' + x)
speech[['president','links','full_links']].head()

,president,links,full_links
0,George Washington,/wiki/George_Washington%27s_First_Inaugural_Ad...,https://en.wikisource.org/wiki/George_Washingt...
1,George Washington,/wiki/George_Washington%27s_Second_Inaugural_A...,https://en.wikisource.org/wiki/George_Washingt...
2,John Adams,/wiki/John_Adams%27_Inaugural_Address,https://en.wikisource.org/wiki/John_Adams%27_I...
3,Thomas Jefferson,/wiki/Thomas_Jefferson%27s_First_Inaugural_Add...,https://en.wikisource.org/wiki/Thomas_Jefferso...
4,Thomas Jefferson,/wiki/Thomas_Jefferson%27s_Second_Inaugural_Ad...,https://en.wikisource.org/wiki/Thomas_Jefferso...


In [0]:
plain_text = []

#for each in range(len(speech)-3,len(speech)):
for each in range(len(speech)):
    
    html = requests.get(speech.full_links.loc[each]).content
    sel = Selector (text = html)
    xpath3 = '//*[@class="mw-parser-output"]//p//text()'
    # Extract text from website + join in 1string + remove unwanted text and new lines
    text = ''.join(sel.xpath(xpath3).extract()).replace('\nThis work is in the public domain in the United States because it is a work of the United States federal government (see 17\xa0U.S.C.\xa0105).\n','')\
.replace('\n',' ')
    text_lines = ''.join(sel.xpath(xpath3).extract()).replace('\nThis work is in the public domain in the United States because it is a work of the United States federal government (see 17\xa0U.S.C.\xa0105).\n','')
    plain_text.append(text)   

In [0]:
def hm_words(x):
    words = len(x.split())
    return words

def hm_ch_words(x):
    words_ch = round(np.mean([len(x) for x in x.split()]),2)
    return words_ch

def n_sentences(x):
    sent = len(x.split('.'))
    return sent


In [45]:
speeches = pd.DataFrame(plain_text, columns = ['speeches'])
speeches['characters'] = speeches.speeches.apply(len)
speeches['words'] = speeches.speeches.apply(hm_words)
speeches['ch_in_words'] = speeches.speeches.apply(hm_ch_words)
speeches['sentences'] = speeches.speeches.apply(n_sentences)

speeches['ch_in_sent'] = round(speeches['characters'] / speeches['sentences'],1)
speeches = pd.concat([speech['president'],speeches.iloc[:,1:]], axis = 1)
speeches['n'] = speeches.groupby('president').cumcount() + 1
speeches.tail(5)

,president,characters,words,ch_in_words,sentences,ch_in_sent,n
53,George W. Bush,9005,1586,4.68,84,107.2,1
54,George W. Bush,11918,2083,4.72,104,114.6,2
55,Barack Obama,13140,2328,4.64,109,120.6,1
56,Barack Obama,11954,2114,4.65,86,139.0,2
57,Donald Trump,8413,1456,4.78,96,87.6,1


In [53]:
double = speeches[speeches['n'] == 2].president
double_stats = speeches[(speeches['president'].isin(double))&speeches['n'].isin([1,2])].groupby('n').agg({'words':['mean','std'],'sentences':['mean','std'],\
                                                                                           'ch_in_sent':['mean','std'],'characters':['mean','std'],\
                                                                                           'ch_in_words':['mean','std']}).round(1).T
double_stats['diff'] = double_stats[2]-double_stats[1]
double_stats

n                       1        2    diff
words       mean   2084.3   1827.1  -257.2
            std     881.8    912.6    30.8
sentences   mean     80.4     71.4    -9.0
            std      40.5     37.3    -3.2
ch_in_sent  mean    177.6    161.5   -16.1
            std      76.5     55.7   -20.8
characters  mean  12090.0  10595.5 -1494.5
            std    5200.0   5385.9   185.9
ch_in_words mean      4.8      4.8     0.0
            std       0.2      0.2     0.0

In [54]:
double_stats2 = speeches[(speeches['president'].isin(double))&speeches['n'].isin([1,2])].groupby('n').agg({'words':['median','std'],'sentences':['median','std'],\
                                                                                           'ch_in_sent':['median','std'],'characters':['median','std'],\
                                                                                           'ch_in_words':['median','std']}).round(1).T
double_stats2['diff'] = double_stats2[2]-double_stats2[1]
double_stats2

n                         1        2   diff
words       median   1718.0   1800.0   82.0
            std       881.8    912.6   30.8
sentences   median     87.0     70.0  -17.0
            std        40.5     37.3   -3.2
ch_in_sent  median    161.8    139.4  -22.4
            std        76.5     55.7  -20.8
characters  median  10120.0  10582.0  462.0
            std      5200.0   5385.9  185.9
ch_in_words median      4.8      4.8    0.0
            std         0.2      0.2    0.0

In [57]:
speeches.groupby(['president']).median().sort_values('words', ascending = False).head()

,characters,words,ch_in_words,sentences,ch_in_sent,n
president,,,,,,
William Henry Harrison,49652.0,8424.0,4.89,213.0,233.1,1.0
William Howard Taft,32119.0,5428.0,4.92,161.0,199.5,1.0
James K. Polk,28695.0,4793.0,4.98,147.0,195.2,1.0
Benjamin Harrison,26195.0,4392.0,4.96,153.0,171.2,1.0
Calvin Coolidge,23946.0,4056.0,4.90,197.0,121.6,1.0
